# Video: Surprising View Transformations

This video shows off some more surprising data transformations that can be implemented with views.
The resulting arrays look much more different from original data than the examples that we have looked at so far, but still can be created instantly using views.


Script:
* The views that we have looked at so far have kept the same data in the same order, but shrank the ranges, or otherwise tweaked how we see the same data.
* NumPy views are also capable of reordering data along an axis, and reordering axes themselves.
* I will run through a few examples of these now.

In [ ]:
import numpy as np

In [ ]:
x = np.arange(20).reshape(4, 5)
x

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19]])

Script:
* Since we will be looking at a number of examples, I'll write a quick function to print out each array and its strides so we can see what kind of trick was used to rearrange the data.

In [ ]:
def check(a):
    print("SHAPE")
    print(a.shape)
    print("DATA")
    print(a)
    print("STRIDES")
    print(a.strides)

check(x)

SHAPE
(4, 5)
DATA
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]
STRIDES
(40, 8)


Script:
* Let's look at the transpose operation first.
* This is a transformation usually done to two dimensional arrays which basically swaps the rows and columns.

In [ ]:
check(np.transpose(x))

SHAPE
(5, 4)
DATA
[[ 0  5 10 15]
 [ 1  6 11 16]
 [ 2  7 12 17]
 [ 3  8 13 18]
 [ 4  9 14 19]]
STRIDES
(8, 40)


Script:
* Compare the data between those to arrays printed out.
* The original x has a first row 0, 1, 2, 3, 4.
* The transposed array has a first column 0, 1, 2, 3, 4.
* How did this work?
* The strides were swapped.
* So moving along a row of the new array was the same as moving along a column of the old array.
* And moving along a column of the new array was the same as moving along a row of the old array.
* Pause and think about that for a minute if that did not click in your head.
* Ok, I hope that makes a little sense to you now.
* In linear algebra, transposing matrices, or two-dimensional arrays is a very common operation.
* It has it's own syntax with a superscript T, and numpy makes it available on every array as a dot T attribute.

In [ ]:
check(x.T)

SHAPE
(5, 4)
DATA
[[ 0  5 10 15]
 [ 1  6 11 16]
 [ 2  7 12 17]
 [ 3  8 13 18]
 [ 4  9 14 19]]
STRIDES
(8, 40)


Script:
* Before I learned about views, I was surprised how casually transposes were used in modeling code.
* I wondered why they kept on rewriting the arrays with the transposed layout, instead of rewriting the expressions to not need the transposes.
* Turns out the answer was views.
* The transposes were basically free.
* NumPy has a few other functions like the transposes which rearrange the axes more generally, and like transpose, they are basically free because they are just creating views of existing data.
* Let's look at another kind of transformation.

In [ ]:
check(np.flipud(x))

SHAPE
(4, 5)
DATA
[[15 16 17 18 19]
 [10 11 12 13 14]
 [ 5  6  7  8  9]
 [ 0  1  2  3  4]]
STRIDES
(-40, 8)


Script:
* The numpy function flip you dee flips a two dimensional matrix upside down.
* How does that work?
* It uses a negative stride!
* The original x array has a stride of 40 to move between rows.
* The flipped upside down array has a stride of negative 40 to move between rows, so it is moving the same amount, but in the opposite direction through memory.
* Meanwhile, the stride to move within a row is still the same.

In [ ]:
check(np.fliplr(x))

SHAPE
(4, 5)
DATA
[[ 4  3  2  1  0]
 [ 9  8  7  6  5]
 [14 13 12 11 10]
 [19 18 17 16 15]]
STRIDES
(40, -8)


Script:
* The numpy function flip ell are flips a two dimensional matrix from left to right.
* And as you probably guessed from flipping upside down, this also uses a negative stride for the flipped axis.
* So let's sum up the stride tricks we have seen.
* A stride of zero repeats data.
* Multiplying a stride by a positive integer lets you skip or step through data faster.
* Flipping the sign of a stride lets you go backwards.
* While preparing this video, I wondered if there is a NumPy function that will let you use any stride that you want.
* There is indeed such a function, numpy dot lib dot stride tricks dot as strided, but it comes with some scary warnings because it is very easy to go outside of your data in memory and crash or corrupt your program.
* So I won't show you that.
* But I will show you this other function that I found.
* This function is for making a sliding window view of an array.
* Here is our example array x again.

In [ ]:
x

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19]])

Script:
* If I ask for a 2 by 2 sliding window of x, it will start in the top left corner above, and make an array of 0, 1, 5 and 6.
* Then the next one will be to the right by one, so 1, 2, 6 and 7.
* The window will move all the way to the right, then go back to the left and move down one row.
* Let's look at it now.

In [ ]:
check(np.lib.stride_tricks.sliding_window_view(x, (2, 2)))

SHAPE
(3, 4, 2, 2)
DATA
[[[[ 0  1]
   [ 5  6]]

  [[ 1  2]
   [ 6  7]]

  [[ 2  3]
   [ 7  8]]

  [[ 3  4]
   [ 8  9]]]


 [[[ 5  6]
   [10 11]]

  [[ 6  7]
   [11 12]]

  [[ 7  8]
   [12 13]]

  [[ 8  9]
   [13 14]]]


 [[[10 11]
   [15 16]]

  [[11 12]
   [16 17]]

  [[12 13]
   [17 18]]

  [[13 14]
   [18 19]]]]
STRIDES
(40, 8, 40, 8)


Script:
* Why oh why would we want to use sliding windows like this?
* Most of the state of the art vision algorithms from 2010 until 2020 were based on convolutional neural networks which use sliding windows like this, and some of the state of the art still uses this kind of architecture.
* The first of those algorithms, a neural network known as AlexNet, was also the first to successfully use deep learning, directly leading to all the improvements that we have seen in AI recently.
* Backing off the hyperbole, these sliding windows are useful for repeating an analysis across an image, and this multi-dimensional structure works nicely with modern neural network architectures.
* I hope these last few examples hopefully gave you a strong impression of how flexible NumPy views can be.

**Code Notes:**
* This video was meant to show you the variety of transformations that can be done instantly with views.
* You do not need to learn all these functions now, but here is a list of the functions just used or referenced if you are curious to read more about them.
  * [`numpy.transpose`](https://numpy.org/doc/stable/reference/generated/numpy.transpose.html)
  * [`numpy.moveaxis`](https://numpy.org/doc/stable/reference/generated/numpy.moveaxis.html)
  * [`numpy.swapaxes`](https://numpy.org/doc/stable/reference/generated/numpy.swapaxes.html)
  * [`numpy.flipud`](https://numpy.org/doc/stable/reference/generated/numpy.flipud.html)
  * [`numpy.fliplr`](https://numpy.org/doc/stable/reference/generated/numpy.fliplr.html)
  * [`numpy.lib.stride_tricks.sliding_window_view`](https://numpy.org/doc/stable/reference/generated/numpy.lib.stride_tricks.sliding_window_view.html)
